In [6]:
#import modules
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import os, shutil

In [ ]:
# 데이터 가져오기
# (train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# train_images = train_images.reshape((60000, 28, 28, 1))
# test_images = test_images.reshape((10000, 28, 28, 1))

In [ ]:
# 데이터 전처리

In [7]:
# 모델
model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# 훈련 설정
model.compile(  
  optimizer='sgd',
  loss='mse',
  metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

In [ ]:
# 훈련

In [ ]:
# 저장
model.save('cats_and_dogs_small_1.h5')

In [ ]:
# 그래프